In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.optimize import minimize
from sklearn.preprocessing import StandardScaler

In [3]:
df = pd.read_csv('/Users/dominicprenovost/Programmation/TP2-PF-management/48_Industry_Portfolios.CSV', header=6)
df = df.rename(columns={'Unnamed: 0': 'Date'})

df_48ind = df.iloc[:1171].copy()
df_48ind['Date'] = pd.to_datetime(df_48ind['Date'], format='%Y%m')
df_48ind.set_index('Date', inplace=True)
df_48ind = df_48ind.apply(pd.to_numeric, errors='coerce')

df_numfirm = df.iloc[2564-20:3735-20].copy()
df_numfirm['Date'] = pd.to_datetime(df_numfirm['Date'], format='%Y%m')
df_numfirm.set_index('Date', inplace=True)
df_numfirm = df_numfirm.apply(pd.to_numeric, errors='coerce')

df_avgsize = df.iloc[3739-22:4910-22].copy()
df_avgsize['Date'] = pd.to_datetime(df_avgsize['Date'], format='%Y%m')
df_avgsize.set_index('Date', inplace=True)
df_avgsize = df_avgsize.apply(pd.to_numeric, errors='coerce')


## 1) La capitalisation boursière en tant que taille moyenne de l'entreprise x nombre d'entreprises

In [31]:
market_cap = df_numfirm.multiply(df_avgsize, axis=0)

## 2) Le rapport entre la valeur comptable (BE : book equity) et la valeur de marché (ME : market equity), c'est-à-dire le rapport book-to-market, en utilisant les données "Sum of BE / Sum of ME".

### (Remarque : les données "Somme de BE / Somme de ME" sont annuelles et doivent être converties en données mensuelles en supposant que le ratio reste constant entre juillet de l'année s et juin de l'année s + 1. Par exemple, le ratio de 1926 est le ratio qui doit être utilisé pour la période allant de juillet 1926 à juin 1927).

In [22]:
df_BtoM = df.iloc[4890:4988].copy()
df_BtoM = df_BtoM.apply(pd.to_numeric, errors='coerce')

df_BtoM = df_BtoM.loc[df_BtoM.index.repeat(12)].reset_index(drop=True)

df_BtoM['Date'] = pd.to_datetime(df_BtoM['Date'], format='%Y')

df_BtoM = df_BtoM.drop('Date', axis=1)

df_BtoM = pd.DataFrame(data = df_BtoM.iloc[5:].values, index = df_48ind.index, columns = df_48ind.columns)

df_BtoM.replace(-99.99, np.nan, inplace = True)
df_BtoM.replace(-999, np.nan, inplace = True)
df_BtoM.dropna(inplace = True)

## 3) Le momentum de chaque industrie est le rendement moyen de cette industrie au cours des 12 derniers mois, y compris le mois t.

In [6]:
# Calculer la moyenne mobile sur 12 mois pour chaque industrie
df_mom = df_48ind.rolling(window=12).mean()

df_mom.replace(-99.99, np.nan, inplace = True)
df_mom.replace(-999, np.nan, inplace = True)
df_mom.dropna(inplace = True)

## Pour chaque mois t, standardiser chaque caractéristique de manière transversale pour avoir une moyenne nulle et un écart-type unitaire pour toutes les actions à la date t, comme expliqué dans Brandt et al. (2009).

In [38]:
# Créer un objet StandardScaler
scaler = StandardScaler()

def standardize(df):
    # Standardiser les données
    df_scaled = scaler.fit_transform(df)
    
    df_standardized = pd.DataFrame(df_scaled, columns=df.columns, index=df.index)
    
    return df_standardized

standardized_market_caps = standardize(market_cap)
standardized_BtoM = standardize(df_BtoM)
standardized_mom = standardize(df_mom)

In [52]:
standardized_mom

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1969-07-01,-0.689567,-0.988299,-0.637683,-0.604571,-0.794286,-0.515915,-0.957432,-1.427232,-0.469193,-1.271476,...,-0.268799,-1.373947,-1.443476,-0.454795,-0.028210,-0.959620,-0.679834,0.458288,-0.827547,-0.498115
1969-08-01,-0.432832,-0.781008,-0.599273,-0.667004,-0.456205,-0.368395,-0.845416,-1.030329,-0.304049,-1.365325,...,-0.192402,-1.110734,-1.133417,-0.145353,0.112186,-0.492252,-0.904375,0.659333,-0.625786,-0.134649
1969-09-01,-0.833021,-1.174734,-0.301005,-1.002437,-0.531072,-0.694606,-1.354300,-1.259834,-0.458518,-1.620113,...,-0.231146,-1.759102,-1.550324,-0.585914,-0.327849,-0.987718,-1.868762,0.085440,-0.987149,-1.044523
1969-10-01,-0.285915,-0.734873,-0.017911,-0.056299,-0.019713,-0.451398,-0.855380,-1.051717,-0.106882,-1.410053,...,-0.239877,-1.519681,-1.175857,-0.277521,-0.215249,-0.613074,-1.512808,0.167852,-0.731184,-0.618599
1969-11-01,-0.689072,-1.136815,-0.487837,-0.512625,-0.134133,-0.953400,-1.245032,-1.247906,-0.697128,-1.712365,...,-0.728819,-2.309322,-2.072581,-0.729097,-0.746683,-1.166142,-1.803055,-0.482471,-1.413042,-0.974409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-01,-0.849346,-0.679258,-0.163963,-0.353138,0.020781,-0.001227,0.672302,0.840668,0.124821,0.393834,...,0.231057,0.524065,0.724940,0.074404,0.325137,-0.041743,-0.094683,0.304098,0.178247,0.804640
2023-10-01,-1.817418,-1.705601,-0.520083,-1.136951,-0.735899,-0.663072,0.252069,0.395231,-0.233722,0.127464,...,-0.267162,-0.126781,-0.002026,0.015138,-0.253410,-0.803674,-0.577394,-0.226928,-0.587929,0.298931
2023-11-01,-2.049913,-1.623443,-0.658073,-1.099492,-0.774981,-0.641687,0.382640,0.075653,-0.513147,-0.147292,...,-0.272073,-0.020207,0.141769,0.225978,-0.174260,-0.429499,-0.446498,-0.257500,-0.376704,0.283216


In [43]:
#len(standardized_market_caps)
#len(standardized_BtoM)
len(standardized_mom)

655

In [ ]:
# Divisez chaque market cap par la somme totale correspondante
market_weights = market_cap.div(market_cap.sum(axis=1), axis=0)

optimal_weights = market_weights + (1 / len(market_weights.columns)) * (theta1 * standardized_market_caps + theta2 * standardized_BtoM + theta3 * standardized_mom)

In [ ]:

# Calculer la fonction à maximiser
# Vous pouvez remplacer les opérations par votre propre formule
function_to_maximize = (1/T) * np.sum(u * np.sum(wi_t + (1/N) * (ß_MC + ß_BM + ß_MOM)) * (1 + r))

# Afficher le résultat
print("Fonction à maximiser :", function_to_maximize)


# Code original

In [ ]:
import numpy as np
from scipy.optimize import minimize

def u(x, gamma):
    return (x ** (1 - gamma) - 1) / (1 - gamma)

# Fonction à maximiser
def function_to_maximize(beta, MC, BM, MOM, r, T, N, gamma):
    wi_t = np.random.rand(N) # Supposons que vous avez une fonction pour calculer wi_t
    return -(1/T) * np.sum(u(np.sum(wi_t + (1/N) * (beta[0] * MC + beta[1] * BM + beta[2] * MOM)), gamma) * (1 + r))

# Données
T = 100  # Nombre d'observations
N = 10   # Nombre d'actifs
MC = np.random.rand(T, N)  # Exemple de données de market cap
BM = np.random.rand(T, N)  # Exemple de données de book-to-market ratio
MOM = np.random.rand(T, N) # Exemple de données de momentum
r = np.random.rand(T)       # Exemple de données de rendement
gamma = 0.5  # Paramètre de la fonction d'utilité CRRA

# Initialisation des coefficients beta
initial_beta = np.array([0.5, 0.5, 0.5])

# Optimisation des coefficients beta
result = minimize(function_to_maximize, initial_beta, args=(MC, BM, MOM, r, T, N, gamma), method='SLSQP')

# Résultats
optimal_beta = result.x
maximized_value = -result.fun

print("Coefficients β optimaux :", optimal_beta)
print("Valeur maximisée de la fonction :", maximized_value)


# ******

In [51]:
def maximize_utility(MC, BM, MOM, r, T, N, gamma):
    def u(x, gamma):
        return (x ** (1 - gamma) - 1) / (1 - gamma)

    def function_to_maximize(beta, MC, BM, MOM, r, T, N, gamma):
        wi_t = market_weights  # Assurez-vous d'avoir la bonne façon de calculer wi_t
        # Calcul de la somme pondérée avec les coefficients beta
        weighted_sum = np.sum(beta[0] * MC[:T, :] + beta[1] * BM[:T, :] + beta[2] * MOM[:T, :], axis=1)
        return -(1/T) * np.sum(u(np.sum(wi_t[:T, :] + (1/N) * weighted_sum, axis=1), gamma) * (1 + r))

    # Initialisation des coefficients beta
    initial_beta = np.array([0.5, 0.5, 0.5])

    # Optimisation des coefficients beta
    result = minimize(function_to_maximize, initial_beta, args=(MC, BM, MOM, r, T, N, gamma), method='SLSQP')

    # Résultats
    optimal_beta = result.x
    maximized_value = -result.fun

    return optimal_beta, maximized_value


maximize_utility(MC, BM, MOM, r, 655, 48, 5)

InvalidIndexError: (slice(None, 655, None), slice(None, None, None))